In [1]:
import keras
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.metrics import categorical_crossentropy
from keras.models import load_model
import pandas as pd
import numpy as np
from tensor2tensor.utils.adafactor import AdafactorOptimizer
import functools

Using TensorFlow backend.


In [2]:
net = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3))

In [3]:
x = net.output
x = Flatten()(x)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(20, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)

In [4]:
count=0
for layer in net_final.layers:
    count+=1
print(count)

92


In [5]:
print(net_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [6]:
for layer in net_final.layers:
    layer.trainable = True

In [7]:
optimizerAdafactor = AdafactorOptimizer(learning_rate=None,
                                        multiply_by_parameter_scale=True,
                                        decay_rate=None,
                                        factored=True,
                                        simulated_quantize_bits=None, 
                                        use_locking=False)

In [8]:
net_final.compile(optimizer = optimizerAdafactor, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
V1 = np.load("C:/Users/cloudadavncenew0709/Desktop/numpydataKeras_20_new_val/imgonehot_new_val_2000.npy")
V2 = np.load("C:/Users/cloudadavncenew0709/Desktop/numpydataKeras_20_new_val/labelonehot_new_val_2000.npy")

In [ ]:
weights_file = 'C:/Users/cloudadavncenew0709/Desktop/MobileNet_Adafactor_SaveBest_02.h5'
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='auto', verbose=2)
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_loss', save_best_only=True, verbose=2)
callbacks = [model_checkpoint]

In [ ]:
output = []

In [10]:
for i in range(5):#訓練5週期
    for j in range(100):#100,000張分100批訓練
        X = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/imgonehot_"+str((j+1)*1000)+".npy" )
        Y = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/labelonehot_"+str((j+1)*1000)+".npy" )
        net_final.fit(x = X, y = Y,
                      epochs = 1,#訓練週期
                      validation_data=(V1, V2),
                      callbacks = callbacks,
                      verbose = 1)#顯示訓練過程，1为输出进度条记录，2为每个epoch输出一行记录
        loss = net_final.history.history['loss'][0]
        acc = net_final.history.history['acc'][0]
        val_loss = net_final.history.history['val_loss'][0]
        val_acc = net_final.history.history['val_acc'][0]
        epoch = float(i) + (float(j+1)/100.00)
        output.append([loss, acc, val_loss, val_acc, epoch])
        print(output[-1])
        print('Done training ', (j+1)*1000 ,' images in Epoch : ', i)
    print('Done training 100000 images, Epoch ', i ,'  -------------')
output_df = pd.DataFrame(output, columns=['loss', 'acc', 'val_loss', 'val_acc', 'epoch'])
output_df.to_csv("C:/Users/cloudadavncenew0709/Desktop/MobileNet_Adafactor_SaveBest_data.csv")

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 29s 29ms/step - loss: 3.6202 - acc: 0.0950 - val_loss: 3.1730 - val_acc: 0.0985
Done training  1000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 2.9921 - acc: 0.1400 - val_loss: 3.0497 - val_acc: 0.1015
Done training  2000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 2.9078 - acc: 0.1600 - val_loss: 3.4064 - val_acc: 0.1030
Done training  3000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 2.8019 - acc: 0.1750 - val_loss: 3.2217 - val_acc: 0.0995
Done training  4000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================]

1000/1000 [==============================] - 21s 21ms/step - loss: 2.7136 - acc: 0.2560 - val_loss: 3.7222 - val_acc: 0.1860
Done training  73000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 2.6447 - acc: 0.2400 - val_loss: 3.1917 - val_acc: 0.1750
Done training  74000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 2.5594 - acc: 0.2700 - val_loss: 3.0042 - val_acc: 0.2295
Done training  75000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 2.6289 - acc: 0.2360 - val_loss: 5.0526 - val_acc: 0.1375
Done training  76000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 2.6262 - acc: 0.2930 - val_lo

1000/1000 [==============================] - 21s 21ms/step - loss: 3.0504 - acc: 0.0620 - val_loss: 5.7054 - val_acc: 0.0560
Done training  45000  images in Epoch :  1
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0231 - acc: 0.0610 - val_loss: 3.0158 - val_acc: 0.0510
Done training  46000  images in Epoch :  1
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0014 - acc: 0.0380 - val_loss: 3.0419 - val_acc: 0.0730
Done training  47000  images in Epoch :  1
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0168 - acc: 0.0520 - val_loss: 3.5897 - val_acc: 0.0905
Done training  48000  images in Epoch :  1
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0962 - acc: 0.0650 - val_lo

1000/1000 [==============================] - 21s 21ms/step - loss: 3.0050 - acc: 0.0440 - val_loss: 3.0027 - val_acc: 0.0500
Done training  17000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.0078 - acc: 0.0460 - val_loss: 3.0025 - val_acc: 0.0500
Done training  18000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.0058 - acc: 0.0570 - val_loss: 3.0023 - val_acc: 0.0500
Done training  19000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.0074 - acc: 0.0370 - val_loss: 3.0016 - val_acc: 0.0500
Done training  20000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 2.9966 - acc: 0.0530 - val_lo

1000/1000 [==============================] - 21s 21ms/step - loss: 3.0200 - acc: 0.0390 - val_loss: 3.0087 - val_acc: 0.0500
Done training  89000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0109 - acc: 0.0490 - val_loss: 3.0096 - val_acc: 0.0500
Done training  90000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0055 - acc: 0.0560 - val_loss: 3.0146 - val_acc: 0.0500
Done training  91000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.0135 - acc: 0.0380 - val_loss: 3.0125 - val_acc: 0.0500
Done training  92000  images in Epoch :  2
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0072 - acc: 0.0500 - val_lo

1000/1000 [==============================] - 20s 20ms/step - loss: 3.0325 - acc: 0.0470 - val_loss: 3.0277 - val_acc: 0.0500
Done training  61000  images in Epoch :  3
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.0305 - acc: 0.0480 - val_loss: 3.0266 - val_acc: 0.0500
Done training  62000  images in Epoch :  3
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.0211 - acc: 0.0600 - val_loss: 3.0358 - val_acc: 0.0500
Done training  63000  images in Epoch :  3
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0323 - acc: 0.0450 - val_loss: 3.0445 - val_acc: 0.0500
Done training  64000  images in Epoch :  3
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.0295 - acc: 0.0410 - val_lo

1000/1000 [==============================] - 21s 21ms/step - loss: 3.1520 - acc: 0.0450 - val_loss: 3.2394 - val_acc: 0.0500
Done training  33000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.1966 - acc: 0.0350 - val_loss: 3.2473 - val_acc: 0.0500
Done training  34000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.1429 - acc: 0.0650 - val_loss: 3.2271 - val_acc: 0.0500
Done training  35000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.1448 - acc: 0.0540 - val_loss: 3.2200 - val_acc: 0.0500
Done training  36000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 20s 20ms/step - loss: 3.1635 - acc: 0.0460 - val_lo

In [11]:
# 將模型儲存至 HDF5 檔案中
net_final.save('C:/Users/cloudadavncenew0709/Desktop/MobileNet_Adafactor_02.h5')
print('Saved Succeed--------------')

Saved Succeed--------------


C:\Users\cloudadavncenew0709\Anaconda3\envs\keras35\lib\site-packages\keras\engine\saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


# 計算top-1、top-3 accuracy

In [21]:
for layer in net_final.layers:
    layer.trainable = True

In [22]:
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

net_final.compile(optimizer = optimizerAdafactor, loss='categorical_crossentropy', metrics = ['accuracy', top3_acc])

In [23]:
for i in range(2):#訓練5週期
    X = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/imgonehot_"+str((i+1)*1000)+".npy" )
    Y = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/labelonehot_"+str((i+1)*1000)+".npy" )
    net_final.fit(x = X, y = Y,
                    epochs = 1,
                    validation_data=(V1, V2),
                    verbose = 1)

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 29s 29ms/step - loss: 3.2579 - acc: 0.0510 - top3_acc: 0.1300 - val_loss: 3.4619 - val_acc: 0.0500 - val_top3_acc: 0.1500
Done training  1000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 21s 21ms/step - loss: 3.2134 - acc: 0.0530 - top3_acc: 0.1600 - val_loss: 3.3946 - val_acc: 0.0500 - val_top3_acc: 0.1500
Done training  2000  images in Epoch :  0


KeyboardInterrupt: 

# 載入save_best_model

In [ ]:
import numpy as np
from keras.optimizers import Adam
import keras
import functools

In [ ]:
with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    net_final = load_model('C:/Users/cloudadavncenew0709/Desktop/MobileNet_Adafactor_SaveBest_02.h5')

In [ ]:
for layer in net_final.layers:
    layer.trainable = True

In [ ]:
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

net_final.compile(optimizer = optimizerAdafactor, loss='categorical_crossentropy', metrics = ['accuracy', top3_acc])

In [ ]:
for i in range(2):
    X = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/imgonehot_"+str((i+1)*1000)+".npy" )
    Y = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/labelonehot_"+str((i+1)*1000)+".npy" )
    net_final.fit(x = X, y = Y,
                  epochs = 1,
                  validation_data=(V1, V2),
                  verbose = 1)